In [44]:
from qiskit import QuantumCircuit, transpile
from qiskit_aer import Aer

def create_oracle(number):
    binary = bin(number)[2:]
    oracle = QuantumCircuit(len(binary) + 1)
    
    for i, bit in enumerate(reversed(binary)):
        if bit == '1':
            oracle.cx(i, len(binary))
    
    oracle_inst = oracle.to_instruction()
    oracle_inst.name = "Oracle"
    return oracle_inst

def build_circuit(oracle):
    n = oracle.num_qubits - 1
    
    qc = QuantumCircuit(n+1, n)
    
    for qubit in range(n):
        qc.h(qubit)
    
    qc.x(n)
    qc.h(n)
    qc.barrier()
    
    qc.append(oracle, range(n+1))
    qc.barrier()
    
    for qubit in range(n):
        qc.h(qubit)
    
    for qubit in range(n):
        qc.measure(qubit, qubit)

    print(qc.draw(output='text'))
    
    return qc

def run_algorithm(secret_number):
    oracle = create_oracle(secret_number)
    circuit = build_circuit(oracle)

    simulator = Aer.get_backend('aer_simulator')
    compiled = transpile(circuit, simulator)

    result = simulator.run(compiled).result()

    counts = result.get_counts()
    answer = list(counts.keys())[0]
    return int(answer, 2)


if __name__ == "__main__":
    number = int(input("Введите секретное число: "))
    result = run_algorithm(number)
    print(f"Найдено число: {result}")

     ┌───┐      ░ ┌─────────┐ ░ ┌───┐┌─┐      
q_0: ┤ H ├──────░─┤0        ├─░─┤ H ├┤M├──────
     ├───┤      ░ │         │ ░ ├───┤└╥┘┌─┐   
q_1: ┤ H ├──────░─┤1        ├─░─┤ H ├─╫─┤M├───
     ├───┤      ░ │  Oracle │ ░ ├───┤ ║ └╥┘┌─┐
q_2: ┤ H ├──────░─┤2        ├─░─┤ H ├─╫──╫─┤M├
     ├───┤┌───┐ ░ │         │ ░ └───┘ ║  ║ └╥┘
q_3: ┤ X ├┤ H ├─░─┤3        ├─░───────╫──╫──╫─
     └───┘└───┘ ░ └─────────┘ ░       ║  ║  ║ 
c: 3/═════════════════════════════════╩══╩══╩═
                                      0  1  2 
Найдено число: 7
